In [2]:
# install RISE with https://rise.readthedocs.io/en/stable/installation.html
#!pip3 install -U scikit-learn
# all imports
from sklearn.feature_extraction.text import CountVectorizer
import math 
import matplotlib.pyplot as plt
import numpy as np

# Chapter 6: Specific Problem of Natural Language Processing

## by Ziwei Chen, Stephan Nef, Lukas Bamert and Jan Grau

# Agenda

1. Words to mathematical representation
2. Embedding the problem into already learnt
3. Transformer Encoder
    1. Self-Attention
    2. position-wise Feedforward Networks
    3. Residucal connection and Layer Normalization

## Introduction to NLP
give examples on application, where it is used

## Words to mathematical representation

explain the problems of word to NLP, why we need to transform the words into vectors

Take the formula from chapter 6.1 show it with some explanation in markdown and create a code example. Example is below.

In [4]:
# create CountVectorizer object
vectorizer = CountVectorizer()
corpus = [
          'Text of first Lukas.',
          'Text of the second document made longer.',
          'Number three is new.',
          'This is number four.',
]
# learn the vocabulary and store CountVectorizer sparse matrix in X
X = vectorizer.fit_transform(corpus)# columns of X correspond to the result of this method
vectorizer.get_feature_names_out() == (
    ['document', 'first', 'four', 'is', 'longer',
     'made', 'number', 'of', 'second', 'text',
     'the', 'this', 'three', 'new', 'Lukas'])# retrieving the matrix in the numpy form
print(X.toarray())# transforming a new document according to learn vocabulary
vectorizer.transform(['A new Lukas is three.']).toarray()

[[0 1 0 0 0 1 0 0 0 1 0 1 0 0 0]
 [1 0 0 0 1 0 1 0 0 1 1 1 1 0 0]
 [0 0 0 1 0 0 0 1 1 0 0 0 0 0 1]
 [0 0 1 1 0 0 0 0 1 0 0 0 0 1 0]]


array([[0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1]])

explain advantages/disadvantages of Word Embeddings as calculated above

introduce the formula $x_i = Mt_i$ and $x_i^\star= x_i +p_j$

If found, give code example for the formulas otherwise use an image.

In [12]:
#insert code for formula above or use an image to explain

### Semantic Similarity

explain formula $w_{AB} = v_A'v_B$.

In [9]:
# code example of formula above

## Embed previous learnt methods to NLP



### modelling it as FIR
- draw some relations back to FIR with an example and image

### modelling it as IIR
- draw some relations back to IIR with an example and image

### modelling it as RNN
- draw some relations back to discrete linear systems with an example and image
- introduce the new formula $h_t = H(h_{t-1}, x_{t-1})$ and $y_t = Y(h_t,x_t)$
- explain the differences between classical RNNs and our application
- also with the new formula $h_t = H(h_{t-1}, x_{t})$ and $y_t = Y(h_t)$
- explain the vanishing gradient problem in RNN

## from RNN to LSTMs
- introduce the state formula $c_t = C(h_{t-1}, x_t, c_{t-1})$
- introduce formula 6.13
- introduce the three gates (forget gate, input gate and output gate)

In [10]:
# code to show the Hadamard operator

In [11]:
# code to explain the gates (if easily found)

### improvements for RNNs
- bidirectional sequencing
- example with BERT

![Attention is all you need](attention.jpg)

### Attention

Remember the problem of FIR filters? The length of the filter is too short for the actual input and quality of the data of the input may differ.

There is a solution to this: $\underline{attention}$.

Let $v = [v_1,..., v_n]$ be a sequence of input vectors.

Then we can define a context vector $c$ as $c= \sum_{i=1}^n \alpha_iv_i$.

This can be extrapolated to different context vectors, each describing different contexts $j$:

$$c_j = \sum_{i=1}^n \alpha_{ji}v_i $$


where $\alpha_{ji}$ is an attention weight from input $i$ to output $j$. A good way to achieve this is to use the softmax function:

$$\alpha_{ji} = \frac{e^{g_{ji}}}{\sum_{k=1}^ne^{g_{jk}}}$$

where $g_{ji}$ is using an alignment model to tell about the similarity of two vectors:

$$ g_{ji} = \frac{q_j'k_i}{\sqrt{k}}$$


In [1]:
# a context vector calculation in pratice

# let's try to figure out the market value of newcomer Güven 
# given our scouting DB with current market values of known players

v = {}
v["messi"] = 80
v["lewandowski"] = 40
v["maguire"] = -25

# since we already have an example of cosine similarity g_ji is given here

g = {}
g["güven-messi"] = 0.8
g["güven-lewandowski"] = 0.5
g["güven-maguire"] = -0.8

In [3]:
import math
import numpy as np

# a litte helper
sum_eg = 0
for mv in g:
    sum_eg += math.exp(g[mv])

# calculate attention values
alpha_güven_messi = math.exp(g["güven-messi"])/sum_eg
alpha_güven_lewandowski = math.exp(g["güven-lewandowski"])/sum_eg
alpha_güven_maguire = math.exp(g["güven-maguire"])/sum_eg
c = alpha_güven_messi *v["messi"] + alpha_güven_lewandowski * v["lewandowski"] + alpha_güven_maguire * v["maguire"]
print("expected market value of Güven: ", round(c,2), "Mio CHF")

expected market value of Güven:  53.83 Mio CHF


### Takeaway

We calculate basic attention by querying (Güven aka $q$) to keys (Messi & co, aka $k$) to get a value (market value aka $v$).

### Autocoding out of context

Tell something about chapter 6.6. But probably after transformers are explained.

## Self-Attention

What if we try to calculate our $q$, $k$, and $v$ by ourselves?

This process is called self-attention, where each input vector from $x = [x_1,...,x_n]$ is also query, key and value:

$$x_i = q_i = k_i = v_i$$

However instead of just calculating $c_j = \sum_{i=1}^n \alpha_{ji}v_i$ with $\alpha_{ji} = \frac{e^{g_{ji}}}{\sum_{k=1}^ne^{g_{jk}}}$, it has been proven mathematically beneficial to linearly project these vectors (in our example we had a scalar value) into smaller dimensionalities. For this we use three projection matrices $W^Q$, $W^K$, $W^V$. This will give us the following equations:

$$ q_i^\star = W^Qq_i, k_i^\star = W^Kk_i,v_i^\star = W^Vv_i$$


These $W$ play an essential role in the learning. Since the attention mechanism does not contain trainable parameters. Therefore given fixed inputs vectors, we need to learn the elements of the $W$'s. Also note that having two different matrices $W^Q$ and $W^K$ we will have asymetric relationships between the input vector elements.

In the end we can calculate $c_j =  \sum_{i=1}^n \alpha_{ji}v_i^\star$ for each element $x_i$.

In [9]:
import random
# input vector (maybe take values from Stephan/Ziwei)
x = np.array([np.random.random_sample(8) for x in range(3)])

# we need to set the dimensions
d_model = len(x[0]) # always the length of the input vectors
d_q = d_model // 4 # theoretically freely choosable to linear transform the projection matrix
d_v = d_model // 2 # can be different for the values, but usually not

# generate the three projections matrices
W_Q = np.random.random_sample((d_q, d_model)) # in a trainable model, those would be trained instead of random
W_K = np.random.random_sample((d_q, d_model)) # in a trainable model, those would be trained instead of random
W_V = np.random.random_sample((d_v, d_model)) # in a trainable model, those would be trained instead of random

In [10]:
# calculate context vector c_1 (based on x_1)
c_1 = 0

# iterate over each and every element in input vector
for i in range(len(x)):
    # query is always x_1
    q_star = np.matmul(W_Q, x[0])
    k_star = np.matmul(W_K, x[i])
    g_ji = (np.dot(q_star.transpose(),k_star)) / len(W_K[0])
    # for alpha we need the sum of e^g^jk
    e_g_jk = 0
    for i in range(len(x)):
        g_jk = (np.dot(np.dot(W_Q, x[0]).transpose(),np.dot(W_K, x[i]))) / len(W_K[0])
        e_g_jk += g_jk
    alpha = math.exp(g_ji) / e_g_jk 
    v_i_star = np.matmul(W_V, x[i])
    c_1 += np.dot(alpha,v_i_star) 
print(c_1) # c should be the same size as the value vector

[6.3480872  5.96059901 4.44070195 4.14138009]
